In [ ]:
import xml.sax
import re
from collections import defaultdict, OrderedDict
from os import path, listdir
import sys
import pickle
import time
import heapq
import json, nltk
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [ ]:
#Need to change these below
INPUT_FILE = '/datasets/wiki-dump/wiki-pages/'
TITLES = '/datasets/wiki-dump/title_index/'
BODY = '/datasets/wiki-dump/body_index/'
# INDEX_STAT = '​invertedindex_stat.txt​'
TOTAL_TOKENS = 0
TOTAL_INV_TOKENS = 0
indexMap = defaultdict(list)
file_num = 0
pages = 1

In [ ]:
def split_titles(txt):
    return txt.split('_')

In [ ]:
def body_index(sentences, fstr, n):
    res = defaultdict(list)
    itr = 0
    for article in sentences:
        n_lines = article.split('\n')[:n]
        lst = []
        d = defaultdict(lambda: 0)
        for line in n_lines:
            temp = line.split()[1:]
            for j in temp:
                d[j] += 1
        for word in d.keys():
            string = fstr + '-' + str(itr) + '-' + str(d[word])
            res[word].append(string)
        itr += 1
    del sentences
    return res

In [ ]:
def title_index(titles, fstr):
    res = defaultdict(list)
    itr = 0
    for lst in titles:
        lst = lst.split('_')
        ln = 0
        for word in lst:
            if word.startswith('-LRB'):
                ln += 0.5
            else:
                ln += 1
        # print(ln)
        d = defaultdict(lambda: 0)
        for j in lst:
            d[j] += 1
        for word in d.keys():
            string = fstr + '-' + str(itr) + '-' + str(d[word]) + '-' + str(ln)
            # print(string)
            res[word].append(string)
        itr += 1
    del titles
    return res

In [ ]:
def store_index(ind, st, titles = True):
    index_map_file = []
    for key in sorted(ind.keys()):
        string = key + ':' + ' '.join(ind[key])
        index_map_file.append(string)
    if titles:
        with open(TITLES+st+'.txt' ,"w+") as f:
            f.write('\n'.join(index_map_file))
    else:
        with open(BODY+st+'.txt' ,"w+") as f:
            f.write('\n'.join(index_map_file))

In [ ]:
def wiki_parse():
    print("Starting parser")
    file_num = 0
    total_time = 0
    for i in tqdm(range(1,110)):
        if i < 10:
            fstr = '00' + str(i)
        elif i < 100:
            fstr = '0' + str(i)
        else:
            fstr = str(i)
        train = []
        raw = open('/datasets/wiki-dump/wiki-pages/wiki-'+fstr+'.jsonl', 'r')
        for line in raw:
            obj = json.loads(line)
            train.append(obj)
        raw.close()
        df_wiki = pd.DataFrame(train)
        del train
        title_ind = title_index(df_wiki.id, fstr)
        store_index(title_ind, str(fstr))
        del title_ind
        body_ind = body_index(df_wiki.lines, fstr, 2)
        store_index(body_ind, str(fstr), titles=False)
        del body_ind
        del df_wiki
        # parse_start_time = time.time()
        # xml_parser = xml.sax.make_parser()
        # xml_parser.setFeature(xml.sax.handler.feature_namespaces, 0)
        # handler = xml_handler(parse_start_time, file_num)
        # xml_parser.setContentHandler(handler)
        # xml_parser.parse(INPUT_FILE+'/'+filename)
        # print("Parsing finished")
        # # store_index(file_num)
        # # print("Dumping finished")
        # time_taken = time.time() - parse_start_time
        # total_time += time_taken
        # print("Time taken: ",time_taken)
        # print("Time elapsed: ",total_time)
        # print("Pages: ",pages)
        # total_time += time_taken
        # indexMap = defaultdict(list)
        # file_num += 1

In [ ]:
wiki_parse()

100%|██████████| 109/109 [11:22<00:00,  6.26s/it]


In [ ]:
def merge_index_titles():
    global pages
    file_pointers = []
    for filename in listdir(TITLES):
        if filename.endswith('.txt'):
            print(TITLES+filename)
            file_pointers.append(open(TITLES+filename, 'r'))
    # ind = open('index/index','w+')
    # off = open('index/offset','w+')
    file_pointers_temp = file_pointers[0:40]
    last_file_count = 40
    file_pointers_flag = 1
    loop_count = 0
    while file_pointers_flag:
        print(last_file_count)
        wordpostings = defaultdict(lambda: [])
        words = {}
        heap = []
        wordfilemap = defaultdict(lambda: [])
        curline = {}
        finishflag = 1
        flag = 0
        filecomplete = [0 for i in range(len(file_pointers_temp))]
        ind = open(TITLES+'index_temp'+str(loop_count),'w+')
        off = open(TITLES+'offset_temp'+str(loop_count),'w+')
        for i in range(len(file_pointers_temp)):
            curline[i] = file_pointers_temp[i].readline().strip()
            # print(curline[i])
            word = curline[i].split(':')[0]
            wordfilemap[word].append(i)
            wordpostings[word] += curline[i].split(':')[1].split(" ")
            if word not in heap:
                heapq.heappush(heap,word)
        while (finishflag):
            minword = heapq.heappop(heap)
            string = minword + ':' + str(ind.tell())
            string = string.strip() + '\n'
            off.write(string)
            del string
            string = minword + ":" + " ".join(wordpostings[minword]) + "\n"
            ind.write(string)
            del string
            filenum = wordfilemap[minword]
            # print(wordfilemap)
            wordfilemap.pop(minword)
            wordpostings.pop(minword)
            for num in filenum:
                nextline = file_pointers_temp[num].readline().strip()
                if nextline == '':
                    filecomplete[num] = 1
                else:
                    newword = nextline.split(':')[0]
                    wordpostings[newword] += nextline.split(':')[1].split(" ")
                    # print(wordfilemap[newword])
                    if not wordfilemap[newword]:
                        heapq.heappush(heap,newword)
                        wordfilemap[newword].append(num)
                    else:
                        wordfilemap[newword].append(num)
            for i in range(len(file_pointers_temp)):
                flag = filecomplete[i] + flag
            flag = int(flag/(len(file_pointers_temp)))
            if flag==1:
                finishflag = 0
        for i in range(len(file_pointers_temp)):
            file_pointers_temp[i].close()
        ind.close()
        off.close()
        if file_pointers_flag==2:
            file_pointers_flag = 0
        elif last_file_count+39 >= len(file_pointers):
            file_pointers_temp = file_pointers[last_file_count:]
            file_pointers_flag = 2
        else:
            file_pointers_temp = file_pointers[last_file_count:last_file_count+39]
        file_pointers_temp.append(open(TITLES+'index_temp'+str(loop_count),'r'))
        last_file_count = last_file_count + 39
        loop_count += 1
        wordpostings = None
        words = None
        heap = None
        wordfilemap = None
        curline = None
    return

In [ ]:
merge_index_titles()

/datasets/wiki-dump/title_index/001.txt
/datasets/wiki-dump/title_index/002.txt
/datasets/wiki-dump/title_index/003.txt
/datasets/wiki-dump/title_index/004.txt
/datasets/wiki-dump/title_index/005.txt
/datasets/wiki-dump/title_index/006.txt
/datasets/wiki-dump/title_index/007.txt
/datasets/wiki-dump/title_index/008.txt
/datasets/wiki-dump/title_index/009.txt
/datasets/wiki-dump/title_index/010.txt
/datasets/wiki-dump/title_index/011.txt
/datasets/wiki-dump/title_index/012.txt
/datasets/wiki-dump/title_index/013.txt
/datasets/wiki-dump/title_index/014.txt
/datasets/wiki-dump/title_index/015.txt
/datasets/wiki-dump/title_index/016.txt
/datasets/wiki-dump/title_index/017.txt
/datasets/wiki-dump/title_index/018.txt
/datasets/wiki-dump/title_index/019.txt
/datasets/wiki-dump/title_index/020.txt
/datasets/wiki-dump/title_index/021.txt
/datasets/wiki-dump/title_index/022.txt
/datasets/wiki-dump/title_index/023.txt
/datasets/wiki-dump/title_index/024.txt
/datasets/wiki-dump/title_index/025.txt


In [ ]:
def merge_index_body():
    global pages
    file_pointers = []
    for filename in listdir(BODY):
        if filename.endswith('.txt'):
            print(BODY+filename)
            file_pointers.append(open(BODY+filename, 'r'))
    # ind = open('index/index','w+')
    # off = open('index/offset','w+')
    file_pointers_temp = file_pointers[0:40]
    last_file_count = 40
    file_pointers_flag = 1
    loop_count = 0
    while file_pointers_flag:
        print(last_file_count)
        wordpostings = defaultdict(lambda: [])
        words = {}
        heap = []
        wordfilemap = defaultdict(lambda: [])
        curline = {}
        finishflag = 1
        flag = 0
        filecomplete = [0 for i in range(len(file_pointers_temp))]
        ind = open(BODY+'index_temp'+str(loop_count),'w+')
        off = open(BODY+'offset_temp'+str(loop_count),'w+')
        for i in range(len(file_pointers_temp)):
            curline[i] = file_pointers_temp[i].readline().strip()
            # print(curline[i])
            word = curline[i].split(':')[0]
            wordfilemap[word].append(i)
            wordpostings[word] += curline[i].split(':')[1].split(" ")
            if word not in heap:
                heapq.heappush(heap,word)
        while (finishflag):
            minword = heapq.heappop(heap)
            string = minword + ':' + str(ind.tell())
            string = string.strip() + '\n'
            off.write(string)
            del string
            string = minword + ":" + " ".join(wordpostings[minword]) + "\n"
            ind.write(string)
            del string
            filenum = wordfilemap[minword]
            # print(wordfilemap)
            wordfilemap.pop(minword)
            wordpostings.pop(minword)
            for num in filenum:
                nextline = file_pointers_temp[num].readline().strip()
                if nextline == '':
                    filecomplete[num] = 1
                else:
                    newword = nextline.split(':')[0]
                    wordpostings[newword] += nextline.split(':')[1].split(" ")
                    # print(wordfilemap[newword])
                    if not wordfilemap[newword]:
                        heapq.heappush(heap,newword)
                        wordfilemap[newword].append(num)
                    else:
                        wordfilemap[newword].append(num)
            for i in range(len(file_pointers_temp)):
                flag = filecomplete[i] + flag
            flag = int(flag/(len(file_pointers_temp)))
            if flag==1:
                finishflag = 0
        for i in range(len(file_pointers_temp)):
            file_pointers_temp[i].close()
        ind.close()
        off.close()
        if file_pointers_flag==2:
            file_pointers_flag = 0
        elif last_file_count+39 >= len(file_pointers):
            file_pointers_temp = file_pointers[last_file_count:]
            file_pointers_flag = 2
        else:
            file_pointers_temp = file_pointers[last_file_count:last_file_count+39]
        file_pointers_temp.append(open(BODY+'index_temp'+str(loop_count),'r'))
        last_file_count = last_file_count + 39
        loop_count += 1
        wordpostings = None
        words = None
        heap = None
        wordfilemap = None
        curline = None
    return

In [ ]:
merge_index_body()

/datasets/wiki-dump/body_index/001.txt
/datasets/wiki-dump/body_index/002.txt
/datasets/wiki-dump/body_index/003.txt
/datasets/wiki-dump/body_index/004.txt
/datasets/wiki-dump/body_index/005.txt
/datasets/wiki-dump/body_index/006.txt
/datasets/wiki-dump/body_index/007.txt
/datasets/wiki-dump/body_index/008.txt
/datasets/wiki-dump/body_index/009.txt
/datasets/wiki-dump/body_index/010.txt
/datasets/wiki-dump/body_index/011.txt
/datasets/wiki-dump/body_index/012.txt
/datasets/wiki-dump/body_index/013.txt
/datasets/wiki-dump/body_index/014.txt
/datasets/wiki-dump/body_index/015.txt
/datasets/wiki-dump/body_index/016.txt
/datasets/wiki-dump/body_index/017.txt
/datasets/wiki-dump/body_index/018.txt
/datasets/wiki-dump/body_index/019.txt
/datasets/wiki-dump/body_index/020.txt
/datasets/wiki-dump/body_index/021.txt
/datasets/wiki-dump/body_index/022.txt
/datasets/wiki-dump/body_index/023.txt
/datasets/wiki-dump/body_index/024.txt
/datasets/wiki-dump/body_index/025.txt
/datasets/wiki-dump/body_